In [ ]:
pip install --user -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 63.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account al

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Flatten, Dense, Reshape, Rescaling
from keras.callbacks import EarlyStopping
import os
from sklearn.metrics import mean_squared_error
from numpy import sqrt
import tensorflow
import numpy as np

In [ ]:
learn_rate = 0.001
dense1_neurons = 65
dense2_neurons = 32
epochs = 15

In [ ]:
def read_data(patient):
    data = pd.read_csv("/Datasets/" + patient + "/" + patient + "-ws-training(t+30).csv",  sep=',', header=0)
    return data

In [ ]:
def remove_na(data):
    data.dropna(inplace = True)
    return data

In [ ]:
def split(data):
    train, test = train_test_split(data, test_size=0.30)
    return train, test

def get_label(train, test, val):
    y_train = train["var1(t+6)"]
    train.drop("var1(t+6)", axis = 1, inplace = True)
    y_test = test["var1(t+6)"]
    test.drop("var1(t+6)", axis = 1, inplace = True)
    y_val = val["var1(t+6)"]
    val.drop("var1(t+6)", axis = 1, inplace = True)
    return train.to_numpy(), y_train.to_numpy(), test.to_numpy(), y_test.to_numpy(), val.to_numpy(), y_val.to_numpy()

In [ ]:
def clarke_error_grid(ref_values, pred_values, title_string):

    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {}) (prediction : {}).".format(len(ref_values), len(pred_values))

    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 400 or max(pred_values) > 400:
        print("Input Warning: the maximum reference value {} or the maximum prediction value {} exceeds the normal physiological range of glucose (<400 mg/dl).".format(max(ref_values), max(pred_values)))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print("Input Warning: the minimum reference value {} or the minimum prediction value {} is less than 0 mg/dl.".format(min(ref_values),  min(pred_values)))

    #Clear plot
    plt.clf()

    #Set up plot
    plt.scatter(ref_values, pred_values, marker='o', color='black', s=8)
    plt.title(title_string + " Clarke Error Grid")
    plt.xlabel("Reference Concentration (mg/dl)")
    plt.ylabel("Prediction Concentration (mg/dl)")
    plt.xticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.yticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.gca().set_facecolor('white')

    #Set axes lengths
    plt.gca().set_xlim([0, 400])
    plt.gca().set_ylim([0, 400])
    plt.gca().set_aspect((400)/(400))

    #Plot zone lines
    plt.plot([0,400], [0,400], ':', c='black')                      #Theoretical 45 regression line
    plt.plot([0, 175/3], [70, 70], '-', c='black')
    #plt.plot([175/3, 320], [70, 400], '-', c='black')
    plt.plot([175/3, 400/1.2], [70, 400], '-', c='black')           #Replace 320 with 400/1.2 because 100*(400 - 400/1.2)/(400/1.2) =  20% error
    plt.plot([70, 70], [84, 400],'-', c='black')
    plt.plot([0, 70], [180, 180], '-', c='black')
    plt.plot([70, 290],[180, 400],'-', c='black')
    # plt.plot([70, 70], [0, 175/3], '-', c='black')
    plt.plot([70, 70], [0, 56], '-', c='black')                     #Replace 175.3 with 56 because 100*abs(56-70)/70) = 20% error
    # plt.plot([70, 400],[175/3, 320],'-', c='black')
    plt.plot([70, 400], [56, 320],'-', c='black')
    plt.plot([180, 180], [0, 70], '-', c='black')
    plt.plot([180, 400], [70, 70], '-', c='black')
    plt.plot([240, 240], [70, 180],'-', c='black')
    plt.plot([240, 400], [180, 180], '-', c='black')
    plt.plot([130, 180], [0, 70], '-', c='black')

    #Add zone titles
    plt.text(30, 15, "A", fontsize=15)
    plt.text(370, 260, "B", fontsize=15)
    plt.text(280, 370, "B", fontsize=15)
    plt.text(160, 370, "C", fontsize=15)
    plt.text(160, 15, "C", fontsize=15)
    plt.text(30, 140, "D", fontsize=15)
    plt.text(370, 120, "D", fontsize=15)
    plt.text(30, 370, "E", fontsize=15)
    plt.text(370, 15, "E", fontsize=15)

    #Statistics from the data
    zone = [0] * 5
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A

        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E

        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B

    zone = [x/len(ref_values) for x in zone]
    return plt, zone

In [ ]:
def plot(y_sc_test, y_pred):
    clarke_fig, zone = clarke_error_grid(y_sc_test*1, y_pred*1, 'Clarke Error Grid')
    print("Clarke Error Grid Zones")
    print(zone)
    plt.show()

In [ ]:
def define_model(train):
    multi_step_dense = keras.Sequential([
    Rescaling(scale=1/np.std(train), offset= np.mean(train) ),
    Flatten(),
    Dense(units= dense1_neurons, activation='relu'),
    Dense(units= dense2_neurons, activation='relu'),
    Dense(units=1),
    ])
    return multi_step_dense  

In [ ]:
def prepare_dataset():
    #prepare dataset
    data = read_data("544")
    data = remove_na(data)
    train, test = split(data)
    train, val = split(train)
    return get_label(pd.DataFrame(train, columns = data.columns), pd.DataFrame(test, columns = data.columns), pd.DataFrame(val, columns = data.columns))
    

In [ ]:
def compute():
    train, y_train, test, y_test, val, y_val = prepare_dataset()
    #define and train model
    model = define_model(train)
    earlystopping_cb = EarlyStopping(monitor="val_mse", patience=5)
    model.compile(loss = "mae", optimizer = keras.optimizers.Adam(learning_rate=learn_rate), metrics = "mse")
    model.fit(train, y_train, batch_size = 1, epochs = epochs, validation_data=(val, y_val), shuffle = False, callbacks=[earlystopping_cb])
    y_pred = model.predict(test)
    #plot(y_test, y_pred)
    return model

In [ ]:
def metrics(model): 
    train, y_train, test, y_test, val, y_val = prepare_dataset()
    y_pred = model.predict(test)
    mae = sqrt(mean_squared_error(y_test, y_pred))
    print('Test Set MAE: ', mae)
    return mae

In [ ]:
model = compute()

Epoch 1/15
6199/6199 [==============================] - 22s 3ms/step - loss: 45.5020 - mse: 3537.5291 - val_loss: 43.0927 - val_mse: 3100.0964
Epoch 2/15
6199/6199 [==============================] - 21s 3ms/step - loss: 44.8440 - mse: 3431.8884 - val_loss: 43.0174 - val_mse: 3073.8994
Epoch 3/15
6199/6199 [==============================] - 20s 3ms/step - loss: 44.6896 - mse: 3416.8921 - val_loss: 42.8744 - val_mse: 3082.1035
Epoch 4/15
6199/6199 [==============================] - 28s 5ms/step - loss: 44.5481 - mse: 3397.3154 - val_loss: 42.7368 - val_mse: 3066.4624
Epoch 5/15
6199/6199 [==============================] - 20s 3ms/step - loss: 44.4160 - mse: 3370.6089 - val_loss: 42.5400 - val_mse: 3071.6570
Epoch 6/15
6199/6199 [==============================] - 20s 3ms/step - loss: 44.2418 - mse: 3341.4380 - val_loss: 42.3767 - val_mse: 3024.9163
Epoch 7/15
6199/6199 [==============================] - 24s 4ms/step - loss: 44.0851 - mse: 3319.9817 - val_loss: 42.1320 - val_mse: 2984.4431

In [ ]:
MODEL_DIR = '544'
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tensorflow.saved_model.save(model, export_path)

export_path = 544/1

INFO:tensorflow:Assets written to: 544/1/assets


[I 230309 17:26:10 builder_impl:779] Assets written to: 544/1/assets


In [ ]:
mae = metrics(model)

In [ ]:
print(mae)